In [1]:
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches  as mpatches
import matplotlib
import seaborn as sns
sns.set()

/usr/local/miniconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
from lsst.sims.catalogs.db import CatalogDBObject

In [5]:
import lsst.sims.catUtils.baseCatalogModels as bcm
from lsst.sims.catalogs.instance import InstanceCatalog
from lsst.sims.utils import ObservationMetaData

/Users/rbiswas/soft/LSST_el_capitan/DarwinX86/sims_catalogs/master-gf80675f361+4/python/lsst/sims/catalogs/db/dbConnection.py:439: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/rbiswas/soft/LSST_el_capitan/DarwinX86/sims_catalogs/master-gf80675f361+4/python/lsst/sims/catalogs/db/dbConnection.py:439: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


ImportError: No module named generation.db

In [ ]:
from lsst.sims.catUtils.baseCatalogModels import GalaxyTileObj, GalaxyDiskObj
from lsst.sims.catUtils.baseCatalogModels import SNObj
from lsst.sims.catUtils.mixins import FrozenSNCat

In [ ]:
s  = SNObj()

## ObsMetaData

In [ ]:
from lsst.sims.catUtils.utils import ObservationMetaDataGenerator

In [ ]:
degConv = np.array([1., 1./60., 1./3600.])
raConv = degConv / 24.0 * 360.
centralRA = np.dot(np.array([3., 32., 30]), raConv) #03h 32m 30s
centralDec = np.dot(np.array([-28, 6., 0.]), degConv)
patchRadius = 0.4 * np.sqrt(2) #np.dot(np.array([0.0, 10.0, 0.]), degConv)

In [ ]:
opsimPath = os.path.join('/Users/rbiswas/data/LSST/OpSimData')
opsimDB = os.path.join(opsimPath,'enigma_1189_sqlite.db')

In [ ]:
generator = ObservationMetaDataGenerator(driver='sqlite', database=opsimDB) #database = opsimPath, driver='sqlite')
obsMetaDataResults = generator.getObservationMetaData(limit=1000,
                                                      fieldRA=(centralRA - 2.0, centralRA + 2.0), 
                                                      fieldDec=(centralDec - 2.0, centralDec + 2.0),
                                                      expMJD=(49500., 49690.),
                                                      boundLength=0.05,
                                                      boundType='circle')

In [ ]:
[(obsMetaDataResults.index(x), x.mjd.TAI) for x in obsMetaDataResults if x.bandpass == 'r']

In [ ]:
TwinklesObsMetaDataSmall = obsMetaDataResults[35]
TwinklesObsMetaDataSmall2 = obsMetaDataResults[145]
TwinklesObsMetaDataSmall3 = obsMetaDataResults[390]

In [ ]:
print (TwinklesObsMetaDataSmall.pointingRA, TwinklesObsMetaDataSmall2.pointingRA, TwinklesObsMetaDataSmall3.pointingRA)

In [ ]:
class TSNCopy(InstanceCatalog):
    
    column_outputs = ['raJ2000', 'decJ2000', 'snid', 'Tgaltileid','Tt0', 'Tx0', 'Tx1', 'Tc', 'Tredshift', ]
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'x0':'%8e'}

In [ ]:
Ttsmall = TSNCopy(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall)
Ttsmall.write_catalog('TTwinklesSN.dat')

In [ ]:
TTSNdf = pd.read_csv('TTwinklesSN.dat', sep=',\s+')

In [ ]:
fig, ax = plt.subplots()
ax.hexbin(np.degrees(TTSNdf['#raJ2000']), np.degrees(TTSNdf.decJ2000))
ax.plot(TwinklesObsMetaDataSmall.pointingRA, TwinklesObsMetaDataSmall.pointingDec, 
         'ys', markersize=14)
circ = mpatches.Circle((TwinklesObsMetaDataSmall.pointingRA, TwinklesObsMetaDataSmall.pointingDec), 
         0.05)
circ.set(edgecolor='r', fill=False, lw=2)
#circ.set(face)
ax.add_artist(circ)
ax.set_aspect('equal')
y_formatter = matplotlib.ticker.ScalarFormatter(useOffset=False)
ax.yaxis.set_major_formatter(y_formatter)
#sns.despine()

In [ ]:
fig, ax = plt.subplots()
ax.hexbin(TTSNdf['#raJ2000'], TTSNdf.decJ2000)
ax.plot(np.radians(TwinklesObsMetaDataSmall.pointingRA), np.radians(TwinklesObsMetaDataSmall.pointingDec), 
         'ys', markersize=14)
circ = mpatches.Circle((np.radians(TwinklesObsMetaDataSmall.pointingRA), np.radians(TwinklesObsMetaDataSmall.pointingDec)), 
         np.radians(0.05))
circ.set(edgecolor='r', fill=False, lw=2)
#circ.set(face)
ax.add_artist(circ)
ax.set_aspect('equal')

##  SN Instance Catalogs

In [ ]:
sncatalog = FrozenSNCat(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall, 
                         column_outputs=['snra', 'galtileid', 'sndec', 'snid', 'x0', 't0', 'flux', 'flux_err', 'mag', 'mag_err',
                                         'TmagNorm', 'TsedFilepath', 'adu'])
sncatalog.surveyStartDate = 49500.
sncatalog.writeSedFile = False
sncatalog.prefix = '/Users/rbiswas/tmp/'

In [ ]:
sncatalog2 = FrozenSNCat(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall2, 
                         column_outputs=['snra', 'galtileid', 'sndec', 'snid', 'x0', 't0', 'flux', 'flux_err', 'mag', 'mag_err',
                                         'TmagNorm', 'TsedFilepath', 'adu'])
sncatalog2.surveyStartDate = 49500.
sncatalog2.writeSedFile = False
sncatalog2.prefix = '/Users/rbiswas/tmp/'
sncatalog3 = FrozenSNCat(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall3, 
                         column_outputs=['snra', 'galtileid', 'sndec', 'snid', 'x0', 't0', 'flux', 'flux_err', 'mag', 'mag_err',
                                         'TmagNorm', 'TsedFilepath', 'adu'])
sncatalog3.surveyStartDate = 49500.
sncatalog3.writeSedFile = False
sncatalog3.prefix = '/Users/rbiswas/tmp/'

In [ ]:
sncatalog.suppressDimSN = True
sncatalog2.suppressDimSN = True
sncatalog3.suppressDimSN = True

In [ ]:
_ = sncatalog.write_catalog('newcat.dat', chunk_size=2000)

In [ ]:
_ = sncatalog2.write_catalog('newcat2.dat', chunk_size=2000)

In [ ]:
_ = sncatalog3.write_catalog('newcat3.dat', chunk_size=2000)

In [ ]:
!wc -l newcat.dat
!wc -l newcat2.dat
!wc -l newcat3.dat

##  PhoSim Catalogs

In [ ]:
from lsst.sims.catUtils.exampleCatalogDefinitions.phoSimCatalogExamples import PhoSimCatalogSN, PhoSimCatalogSersic2D

In [ ]:
pc = PhoSimCatalogSN(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall, column_outputs=['EBV'])
pc.writeSedFile = True
pc.surveyStartDate = 49500
pc.suppressDimSN = True

In [ ]:
pc.write_catalog('SNOnlyPhoSimCatalog.dat')

In [ ]:
sncatdf = pd.read_csv('newcat.dat', sep=',\s+')
sncatdf.rename(columns={'#snid':'snid'}, inplace=True)
sncatdf['snid'] = sncatdf.snid.astype(int)
sncatdf.set_index('snid', inplace=True)
sncatdf['DeltaT'] = sncatalog.mjdobs - sncatdf['t0']
sncatdf['SNR'] = sncatdf.flux / sncatdf.flux_err
sncatdf.head()

In [ ]:
sncatdf.query('z < 0.1535 and z> 0.152')

In [ ]:
np.left_shift(6000039790801, 10) + 42

In [ ]:
pc2 = PhoSimCatalogSN(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall2, column_outputs=['EBV'])
pc2.writeSedFile = True
pc2.surveyStartDate = 49500
pc2.suppressDimSN = True

In [ ]:
pc2.surveyStartDate

In [ ]:
pc2.write_catalog('SNPhoSimCatalog2.dat')

In [ ]:
pc3 = PhoSimCatalogSN(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall3, column_outputs=['EBV'])
pc3.writeSedFile = True
pc3.surveyStartDate = 49500
pc3.suppressDimSN = True

In [ ]:
pc3.write_catalog('SNPhoSimCatalog3.dat')

In [ ]:
sncatdf.TmagNorm.min()

In [ ]:
sncatdf.adu.min()

In [ ]:
sncatdf.flux.min()

In [ ]:
sncatdf.flux_err.min()

In [ ]:
fig, ax = plt.subplots()
sncatdf.SNR.hist(bins=np.arange(-0.5, 50., 1.), histtype='step', lw=2., alpha=1., ax=ax)
ax.set_xlabel('SNR')

In [ ]:
fig, ax = plt.subplots()
sncatdf.adu.apply(lambda x: x* 2.3).hist(bins=np.arange(-0.5, 5., 0.05), histtype='step', lw=2., alpha=1., ax=ax, norm)
ax.set_xlabel('Photons')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2);
_ = ax[0].hist(sncatdf.query('SNR > 5.').z, bins=np.arange(0., 1.2, 0.1), histtype='step', lw=2, color='k', normed=True,
               label='highSNR')
_ = ax[0].hist(sncatdf.query('SNR < 5. and SNR > 0.').z, bins=np.arange(0., 1.2, 0.1), histtype='step', lw=2, color='r', 
             normed=True, label='lowSNR')
_ = ax[0].hist(sncatdf.query('SNR > 0.').z, bins=np.arange(0., 1.2, 0.1), histtype='step', lw=2, color='b', 
             normed=True, label='Active')
ax[0].grid(True)
_ = ax[1].hist(sncatdf.query('SNR > 5.').z, bins=np.arange(0., 1.2, 0.1), histtype='step', lw=2, color='k', normed=False, 
               cumulative=-1)
_ = ax[1].hist(sncatdf.query('SNR < 5. and SNR > 0.').z, bins=np.arange(0., 1.2, 0.1), histtype='step', lw=2, color='r', 
               normed=False, cumulative=-1)
_ = ax[1].grid(True)
_ = ax[0].legend(loc='upper left')
_ = ax[0].set_ylabel('PDF')
_ = ax[1].set_ylabel('Num SN at z > ')
_ = fig.suptitle('Redshift Distribution of SN')

##  PhoSim Catalog For galaxy Disks

In [ ]:
from lsst.sims.catUtils.exampleCatalogDefinitions.phoSimCatalogExamples import PhoSimCatalogSN, PhoSimCatalogSersic2D

In [ ]:
from lsst.sims.catUtils.baseCatalogModels import GalaxyDiskObj

In [ ]:
gDisk = GalaxyDiskObj()

In [ ]:
gDisk.objectTypeId

In [ ]:
class GdiscCatalogClass(InstanceCatalog):
    
    column_outputs = ['raJ2000', 'decJ2000', 'galtileid', 'id', 'redshift']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'x0':'%8e'}

In [ ]:

gd = GdiscCatalogClass(db_obj=gDisk, obs_metadata=TwinklesObsMetaDataSmall)
gd.write_catalog('GalaxyDisks.dat')

In [ ]:
phosimGals = PhoSimCatalogSersic2D(db_obj=gDisk, obs_metadata=TwinklesObsMetaDataSmall)

In [ ]:
phosimGals2 = PhoSimCatalogSersic2D(db_obj=gDisk, obs_metadata=TwinklesObsMetaDataSmall2)

In [ ]:
phosimGals3 = PhoSimCatalogSersic2D(db_obj=gDisk, obs_metadata=TwinklesObsMetaDataSmall3)

In [ ]:
phosimGals2.write_catalog('SNPhoSimCatalog2.dat', write_header=False, write_mode='a')

In [ ]:
phosimGals3.write_catalog('SNPhoSimCatalog3.dat', write_header=False, write_mode='a')

In [ ]:
phosimGals.write_catalog('GalDiskPhoSimCatalog.dat')
phosimGals.write_catalog('SNPhoSimCatalog.dat', write_header=False, write_mode='a')

## Keeping only hosts

In [ ]:
galD = pd.read_csv('GalaxyDisks.dat', sep=',\s+');
galD.rename(columns={'#raJ2000': 'raJ2000'}, inplace=True)
galD.head()

In [ ]:
gg = galD.galtileid.copy(deep=True)
gg.sort(inplace=True)
sng = df.galtileid.copy(deep=True)
sng.sort(inplace=True)

In [ ]:
sgg = set(gg.values); print (len(sgg))

In [ ]:
sngg = set(sng.values); print(len(sngg))

In [ ]:
galdiskids = np.left_shift(np.array(list(sgg.intersection(set(sng.values)))), 10) + 27

In [ ]:
len(sng.values)

In [ ]:
gals = pd.read_csv('SNGalDiskCatalog.dat', sep='\s*', skiprows=410).values

In [ ]:
gdiskids = np.left_shift(df.galtileid, 10).values + 27

In [ ]:
gdiskids

In [ ]:
with open('SNGalDiskCatalog.dat') as f:
    contents = f.read()

lines = contents.split('\n')
lines.pop()
# print len(contents)
with open('SNHostFile.dat', 'w') as f:
    for i, line in enumerate(lines):
        if i < 410:
            # print line
            f.write(line+'\n')
        else:
            objint = int(line.split()[1])
            # print i, int(line.split()[1])
            if objint in galdiskids:
                f.write(line+'\n')

In [ ]:
with open('SNGalDiskCatalog.dat') as f:
    contents = f.read()

In [ ]:
contents.split('\n').pop()

In [ ]:
contents.split('\n').pop()

In [ ]:
gals[:, 1].astype(np.int).sort()

In [ ]:
61449608101915 in gals[:, 1]

In [ ]:
galdiskids

In [ ]:
np.left_shift(np.array(list(sgg.intersection(set(sng.values)))), 10) + 27

In [ ]:
SNgaltileIds = df.galtileid

In [ ]:
np.left_shift(SNgaltileIds, 10) + gdiskobjid

In [ ]:
len(phosimGals.column_outputs)

In [ ]:
len(pc.column_outputs)

In [ ]:
pc = PhoSimCatalogSN(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall, column_outputs=['EBV'])
pc.writeSedFile = True
pc.surveyStartDate = 49500
pc.suppressDimSN = True

In [ ]:
from copy import deepcopy

In [ ]:
len(pc.column_outputs)

In [ ]:
for col in phosimGals.column_outputs:
    if not col in pc.column_outputs:
        print col

In [ ]:
pc.write_catalog('SNPhoSimCatalog.dat')

In [ ]:
np.array([3.2, 1.2]) *1.2

In [ ]:
0.1/11.113

In [ ]:
!wc -l SNPhoSimCatalog.dat

In [ ]:
!head -22 SNPhoSimCatalog.dat

In [ ]:
phdf = pd.read_csv('SNPhoSimCatalog.dat', skiprows=18, sep='\s*')

In [ ]:
phdf.rename(columns={})

In [ ]:
phdf.head()

In [ ]:
plt.hist(np.asarray(phdf)[:,1], log=True)

In [ ]:
def phosimOutDF(fname0, fname1):
    
    phosimOut0 = pd.read_csv(fname0, sep='\s+')
    phosimOut1 = pd.read_csv(fname1, sep='\s+')
    phosimOut0.Photons += phosimOut1.Photons
    phosimOut0.

In [ ]:
phosimOuta = pd.read_csv('/Users/rbiswas/soft/phosim_release/output/centroid_lsst_e_203482_R22_S11_E000.txt', 
                         sep='\s+')

In [ ]:
phosimOutb = pd.read_csv('/Users/rbiswas/soft/phosim_release/output/centroid_lsst_e_203482_R22_S11_E001.txt', sep='\s+')

In [ ]:
phosimOuta.Photons = phosimOuta.Photons + phosimOutb.Photons

In [ ]:
phosimOuta['SourceID'] = phosimOuta.SourceID.astype(int)

In [ ]:
phosimOuta['snid'] = np.right_shift(phosimOuta['SourceID'], 10)

In [ ]:
phosimOuta.set_index('snid', inplace=True)

In [ ]:
sncatdf.adu.min()

In [ ]:
sncatdf['ratio'] = sncatdf.adu / phosimOuta.Photons

In [ ]:
sncatdf['Photons'] = phosimOuta.Photons

In [ ]:
sncatdf.query('ratio == 0.')

In [ ]:
fig, ax = plt.subplots()
sns.distplot(sncatdf.ratio.replace([-np.inf, np.inf],np.nan).dropna(), bins=np.arange(0.0, 5., 0.05), 
             rug_kws=False,kde=False, norm_hist=False).set_xlim(0,)
sns.distributions()
#ax.hist(df.ratio.replace([-np.inf, np.inf],np.nan).dropna() ,bins=np.arange(0., 5, 0.05))
ax.axvline(1./2.3, color='k', **{'label':'1/gain', 'linestyle':'dashed'})
ax.set_xlabel('adu/Photons')
ax.legend(loc='best')

In [ ]:
rv = poisson(mu=1)

In [ ]:
x = np.arange(0, 10)

In [ ]:
plt.plot(x, rv.pmf(x), 'bo')

In [ ]:
plt.hist(sncatdf.Photons / sncatdf.adu / 2.3, np.arange(-0.5,5.5), histtype='step')
plt.plot()

In [ ]:
plt.plot(poisson.pmf)

In [ ]:
df.query('ratio > 1000.')[['adu', 'Photons', 'flux', 'DeltaT', 'mag', 'mag_err', 'flux_err']]

In [ ]:
x = df.query('ratio ==0.').Photons.copy(deep=True)

In [ ]:
x.sort(ascending=False); x

In [ ]:
fig.savefig('invGain.png')

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
print len(df.query('ratio > 10000')), df.query('ratio > 10000').ratio.unique()
print len(df.query('ratio < -10000')), df.query('ratio < -10000').ratio.unique()

In [ ]:
df.query('ratio >500.').adu 

In [ ]:
_df = df.query('ratio == 0.')
_dff = df.query('ratio < 0.')
_ = plt.hist(_df.DeltaT / (_df.z + 1.), bins=np.arange(-30, 70., 1.), histtype='step', color='b', alpha=1., lw=2.)
_ = plt.hist(_dff.DeltaT / (_dff.z + 1.), bins=np.arange(-30, 70., 1.), histtype='step', color='r', alpha=1., lw=2.)
plt.axvline(-20., color='k')
plt.axvline(50., color='k')

In [ ]:
_df = df[['z', 'ratio']].replace([-np.inf, np.inf], [-20., 200.]).dropna()
#sns.jointplot(_df.z, _df.ratio, kind='kde')
plt.plot(_df.z, _df.ratio, 'o')

In [ ]:
adus = df.adu.dropna().sum()
photons = phosimOuta.replace([np.inf],np.nan).Photons.dropna().sum()

In [ ]:
photons / adus

In [ ]:
active = df.query('SNR > 0.').replace([np.inf],np.nan).dropna().ratio
bright = df.query('SNR > 5.').replace([np.inf],np.nan).dropna().ratio

In [ ]:
s = 0.05
plt.hist(active, color='r', bins=np.arange(0., 25, s), histtype='step', label='SNR>0', lw=2)
plt.hist(bright, color='k', bins=np.arange(0., 25, s), histtype='step', label='SNR>5', lw=2)
plt.legend(loc='best')
plt.axvline(1./2.3)
plt.xlim(0, 5)

In [ ]:
import lsst.afw.image as afwi

In [ ]:
img0 = afwi.ImageF('/Users/rbiswas/soft/phosim_release/output/lsst_e_203482_f2_R22_S11_E000.fits.gz')
img1 = afwi.ImageF('/Users/rbiswas/soft/phosim_release/output/lsst_e_203482_f2_R22_S11_E001.fits.gz')

In [ ]:
arr0 = img0.getArray()
arr1 = img1.getArray()

In [ ]:
arr = arr0 + arr1
electrons = arr.sum()

In [ ]:
adus  = df.adu.dropna().sum()

In [ ]:
2.3 / np.sqrt(2)

In [ ]:
electrons / adus

In [ ]:
x.replace([-np.inf, np.inf], np.nan, inplace=True)

In [ ]:
x.dropna().hist()

In [ ]:
df.set_index('snid', inplace=True)

In [ ]:
x.dropna()

In [ ]:
df.index.size

In [ ]:
phosimOuta.index.dtype

In [ ]:
6145532174336041 in df.index.values

In [ ]:
df.iat[6145532174336041]

In [ ]:
phosimOuta['snid']  = phosimOuta.index.values

In [ ]:
phosimOuta['snid'] = np.right_shift(phosimOuta['snid'], 10)

In [ ]:
phosimOuta.set_index('snid', inplace=True)

In [ ]:
phosimOuta['adu']/ phosimOuta['Photons']

In [ ]:
phosimOuta.head()

In [ ]:
df.index.min()

In [ ]:
phosimOuta.index.min()

In [ ]:
np.left_shift(6000000189404,10) < phosimOuta.index.min()

In [ ]:
np.right_shift(np.left_shift(6000000189404,10),10)

In [ ]:
id(None)

In [ ]:
x  = None

In [ ]:
id(x)

In [ ]:
y = None

In [ ]:
xx =3

In [ ]:
yy  = 3

In [ ]:
 == y

In [ ]:
x is y

In [ ]:
id(xx)

In [ ]:
id(yy)

# Scratch

In [ ]:
names = s.get_column_names()

for key in names:
    if 'snra' in names[key]:
        print key

print names['TwinkSN']

In [ ]:
class SNCopy(InstanceCatalog):
    
    column_outputs = ['raJ2000', 'decJ2000', 'snid', 'galtileid','t0', 'x0', 'x1', 'c', 'redshift', ]
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'x0':'%8e'}
                        #'x0':'%8e', 'x1':'%8e', 'c':'%8e', 't0':'%8e'}
        
